# Ejemplo 010

**Problema:** Queremos generar el reporte de análisis de sobrepropagación para un conjunto de sitios.

#### Importamos paquetes necesarios

In [14]:
# Importación de paquetes requeridos
import os
import io
import requests
import pandas as pd
import numpy as np

from IPython.display import display, Markdown, Latex

import json

#### Funciones auxiliares

In [15]:
# Función auxiliar para visualizar los datos
def pp(obj):
    print(json.dumps(obj, indent=2))

In [16]:
# La operación queda con la forma siguiente:
# «para cada fila r, convertir el valor de la columna "col" en un tipo numérico (pd.to_numeric),
# ignorando los valores nulos (errors='coerce'), redondeando hacia abajo al entero más cercano
# (np.floor) y almacenándolo como entero de 64 bits (astype('Int64'))»

def pd_to_int(df: pd.DataFrame, col: str) -> int:
    return np.floor(pd.to_numeric(df[col], errors='coerce')).astype('Int64')

#### Parámetros del script

In [17]:
# Parmámetros
iga_api_base_url = 'http://ing-acc-movil01.personal.corp/ingenier@/symfony/public/index.php/api'

#### Test de conectividad

In [18]:
# Llamamos a un endpoint de prueba en la API de IGA para verificar conectividad
x = requests.get(iga_api_base_url + '/public/db-sync-state')
pp(x.json())

{
  "status": "success",
  "data": {
    "Uid": "94175",
    "State": "DB_SYNC_STATE_IDLE",
    "LastSyncTimestamp": "09-01-2025 12:02:12",
    "SyncState": "OK",
    "ProjectsUpdated": "DataManager, Litoral Sur"
  }
}


#### Descarga de reportes

- Debemos descargar el reporte de Análisis de sobrepropagación
- El endpoint usado es /secure/report/overshooting-cell-analysis.
- Los datos devueltos están en formato binario, y pueden ser leídos directamente por Pandas como un archivo .xlsx, y convertidos a un objeto DataFrame

In [29]:
# Parámetros en el payload de la request con que llamamos al endpoint de IGA:
payload = {
  "siteNames": [
    "CPU641",
    "CPU670",
    "CPU080",
    "CPU445",
    "CPU830",
  ],
  "enabledTechBands": [
    "LTE FDD_AWS_2225",
    "LTE FDD_AWS_ClaroRSH_2100",
    "LTE FDD_AWS_MvsRSH_2100",
  ],
  "cellFootprintPercentOfNearestTaSamples": 85,
  "includeNotOpCells": False,
  "excludeUndisputedDominanceCellsDeviations": True,
  "startDate": "2025-01-02T03:00:00.000Z",
  "endDate": "2025-01-08T03:00:00.000Z",
  "lowOvershootingRatioThreshold": 3,
  "highOvershootingRatioThreshold": 6,
  "lowUndershootingRatioThreshold": 0.4,
  "highUndershootingRatioThreshold": 0.2
}

x = requests.post(iga_api_base_url + '/public/report/overshooting-cell-analysis', json=payload)
# En x almacenamos los datos devueltos por el endpoint de IGA.
# Es un archivo xlsx en formato binario

# Usando pandas, con la función read_excel, leemos los datos y los almacenamos
# en memoria como un objeto DataFrame:
with io.BytesIO(x.content) as fh:
    df_raw = pd.read_excel(fh, "Resultados")

display(df_raw)

,Código,Sector,Id Antena,Estado Sector,Tecnolog./Banda,Latitud Antena,Longitud Antena,Azimuth Efectivo,Ancho Horiz.,Radio de Footprint de TA [m],...,TA_Bin_21,TA_Bin_22,TA_Bin_23,TA_Bin_24,TA_Bin_25,TA_Bin_26,TA_Bin_27,TA_Bin_28,TA_Bin_29,TA_Bin_30
0,CPU080,CZOOTPL11,7,Operativo,LTE FDD_AWS_2225,-34.578448,-58.411770,80,61,573.487997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CPU080,CZOOTPL12,8,Operativo,LTE FDD_AWS_2225,-34.578448,-58.411770,220,61,618.286753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CPU080,CZOOTPL13,9,Operativo,LTE FDD_AWS_2225,-34.578448,-58.411770,325,61,478.896547,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CPU445,CGURTPL11,7,Operativo,LTE FDD_AWS_2225,-34.584488,-58.421751,0,61,302.235989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CPU445,CGURTPL12,8,Operativo,LTE FDD_AWS_2225,-34.584488,-58.421751,120,61,532.070124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CPU445,CGURTPL13,9,Operativo,LTE FDD_AWS_2225,-34.584488,-58.421751,240,61,513.774072,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CPU641,CNAROZL11,4,Operativo,LTE FDD_AWS_2225,-34.588715,-58.419571,0,61,467.310609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CPU641,CNAROZL12,5,Operativo,LTE FDD_AWS_2225,-34.588715,-58.419571,120,61,476.199845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CPU641,CNAROZL13,6,Operativo,LTE FDD_AWS_2225,-34.588715,-58.419571,240,61,964.548327,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,CPU670,CNSCAOL11,4,Operativo,LTE FDD_AWS_2225,-34.583521,-58.412864,80,61,537.332583,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df = df_raw[['Código', 'Sector', 'Tecnolog./Banda', 'Radio de Footprint de TA [m]', 'Radio Aprox. de Best Server [m]']]

display(df)

,Código,Sector,Tecnolog./Banda,Radio de Footprint de TA [m],Radio Aprox. de Best Server [m]
0,CPU080,CZOOTPL11,LTE FDD_AWS_2225,573.487997,339
1,CPU080,CZOOTPL12,LTE FDD_AWS_2225,618.286753,430
2,CPU080,CZOOTPL13,LTE FDD_AWS_2225,478.896547,1014
3,CPU445,CGURTPL11,LTE FDD_AWS_2225,302.235989,219
4,CPU445,CGURTPL12,LTE FDD_AWS_2225,532.070124,346
5,CPU445,CGURTPL13,LTE FDD_AWS_2225,513.774072,645
6,CPU641,CNAROZL11,LTE FDD_AWS_2225,467.310609,383
7,CPU641,CNAROZL12,LTE FDD_AWS_2225,476.199845,207
8,CPU641,CNAROZL13,LTE FDD_AWS_2225,964.548327,407
9,CPU670,CNSCAOL11,LTE FDD_AWS_2225,537.332583,401
